# Little Washita Parking Lot Test Example
This workbook illustrates how to setup and parking lot simulation when you are building and testing your domain. In a "Parking lot" simualtion we set the permeability of the subsurface very low and rain on the domain to check that our topographic slopes are setup the way we want and our drainage networks are where we expect them to be. This is a good initial step to do when building a watershed model before you add all the complexity of the subsurface and the land surface model.   

We provide comments here on the steps that are unique to a parking lot simulation. For a more thoroughly documented script refer to the [Little Washita Annoated ParFlow-CLM simulation script](https://github.com/hydroframe/parflow_python_shortcourse/blob/main/exercises/little_washita/LittleWashita_ParFlowCLM_AnnotatedExample%20copy.ipynb). 

For additional resources on ParFlow Keys and input scripts refer to the [ParFlow Manual](https://parflow-docs.readthedocs.io/en/latest/keys.html#)

**Requriements** 
Before you can run this workbook you will need to make sure that you have parflow and pftools installed. Refer to the main readme of this repo for instructions on getting your modeling environment setup. if you haven't do so already.

**Simulation Inputs**
All input files can be found in the `model_inputs` folder. Here we will be using:
1. Solid file: used to define the shape of the domain 
3. Slope files: slope x and slope y files define the topographic slopes of each grid cell in the x and y directions

### 1. Import required libraries and functions 

In [1]:
import os
import numpy as np
from parflow import Run 
import shutil
from parflow.tools.fs import mkdir, cp, get_absolute_path, exists
from parflow.tools.settings import set_working_directory

## 2. Setup your run directory and initialize your model object
Note we are are just copying in the slope files and the solid file. We don't need the CLM inputs, presure file or indicator file for this run. 

In [2]:
# Name your ParFLow run -- note that all of your output files will have this prefix
runname = 'LW_ParkingLot'

# Create a directory in the outputs folder for this run
run_dir = get_absolute_path(f'outputs/{runname}')
mkdir(run_dir)
print(run_dir)

# create your Parflow model object. For starters we are just goin to set the file version and the run directory we'll add more later
# note that the model will run from the run_dir so all input files should be in the run dir or paths should be specified relative to run_dir
model = Run(runname, run_dir)
model.FileVersion = 4

#copy the model inputs for the simulation into the run directory
#NOTE: you dont have to copy everything into the run directory if you don't want, you can also point to input files in other directories in a simulation if you prefer
input_dir= os.path.join(os.getcwd(), 'model_inputs')
files = ['slopex_LW.pfb', 'slopey_LW.pfb', 'LW.pfsol']
for fname in files:
    shutil.copy(os.path.join(input_dir,fname), run_dir)

/Users/reed/Projects/parflow_python_shortcourse/exercises/little_washita/outputs/LW_ParkingLot


## 3. Setup the computational grid
We are going to go ahead and setup the grid they way we would when we are running full simulations here. Alghough the subsurface properties are not going to matter because we will make it all impermeable. If you prefer you can run your parking lot test with just one layer.

In [3]:
# Processor topology: This is the way that the problem will be split across processors if you want to run in parallel
# The domain is divided in x,y and z dimensions by P, Q and R. The total number of processors is P*Q*R.
model.Process.Topology.P = 1
model.Process.Topology.Q = 1
model.Process.Topology.R = 1

#Locate the origin in the domain.
model.ComputationalGrid.Lower.X = 0.0
model.ComputationalGrid.Lower.Y = 0.0
model.ComputationalGrid.Lower.Z = 0.0

# Define the size of each grid cell. The length units are the same as those on hydraulic conductivity, here that is meters.
model.ComputationalGrid.DX = 1000.0
model.ComputationalGrid.DY = 1000.0
model.ComputationalGrid.DZ = 200.0

# Define the number of grid blocks in the domain.
model.ComputationalGrid.NX = 64
model.ComputationalGrid.NY = 32
model.ComputationalGrid.NZ = 10

### 3.2 Geometries

In our main simulations we willuse an indicator to define units in the subsurface. here we will make everything uniformly impermable so we just have a solid file input. 

In [4]:
#Declare the geometries that you will use for the problem
model.GeomInput.Names = "solid_input"

#Define the solid_input geometry.  
#Note the naming convention here GeomInput.{GeomName}.key
model.GeomInput.solid_input.InputType = "SolidFile"
model.GeomInput.solid_input.GeomNames = "domain"
model.GeomInput.solid_input.FileName = "LW.pfsol"

#First set the name for your `Domain` and setup the patches for this domain
model.Domain.GeomName = "domain"
model.Geom.domain.Patches = "top bottom side"

### 3.3 Variable ${\Delta z}$ 
Here too we don't really need this for our test because we are focusing on the surface but we are keeping our full 3D grid so that we have it setup when we move to more complicated problems.

In [5]:
model.Solver.Nonlinear.VariableDz = True
model.dzScale.GeomNames = "domain"
model.dzScale.Type = "nzList"
model.dzScale.nzListNumber = 10

model.Cell._0.dzScale.Value = 5
model.Cell._1.dzScale.Value = 0.5
model.Cell._2.dzScale.Value = 0.25
model.Cell._3.dzScale.Value = 0.125
model.Cell._4.dzScale.Value = 0.05
model.Cell._5.dzScale.Value = 0.025
model.Cell._6.dzScale.Value = 0.005
model.Cell._7.dzScale.Value = 0.003
model.Cell._8.dzScale.Value = 0.0015
model.Cell._9.dzScale.Value = 0.0005

### 3.4 Topographic slopes 
Next we define topographic slopes and values. This is the main input that we want to test with this script.  Slope files are derived from elevaiton maps and this processing, as well as the resolution of the underlying elevation map, can lead to local sinks and inconsitencies in the drainage network that are non-physical.  

In [6]:
model.TopoSlopesX.Type = "PFBFile"
model.TopoSlopesX.GeomNames = "domain"
model.TopoSlopesX.FileName = "slopex_LW.pfb"

model.TopoSlopesY.Type = "PFBFile"
model.TopoSlopesY.GeomNames = "domain"
model.TopoSlopesY.FileName = "slopey_LW.pfb"

## 4. Setup the surface and subsurface properties 
We will treat the subsurface as homogenous for this test. The key here is that we want a very low permeability subsurface so that the rain we apply will runoff and we can evaluate our surface drainage network. 


In [7]:
# Permeability 
model.Geom.Perm.Names = "domain"
model.Geom.domain.Perm.Type = "Constant"
model.Geom.domain.Perm.Value = 0.000001

# Permeability tensor
model.Perm.TensorType = "TensorByGeom"
model.Geom.Perm.TensorByGeom.Names = "domain"
model.Geom.domain.Perm.TensorValX = 1.0
model.Geom.domain.Perm.TensorValY = 1.0
model.Geom.domain.Perm.TensorValZ = 1.0

# Specific Storage
model.SpecificStorage.Type = "Constant"
model.SpecificStorage.GeomNames = "domain"
model.Geom.domain.SpecificStorage.Value = 1.0e-5

# Porosity
model.Geom.Porosity.GeomNames = "domain"
model.Geom.domain.Porosity.Type = "Constant"
model.Geom.domain.Porosity.Value = 0.01

# Relative Permeability
model.Phase.RelPerm.Type =              "VanGenuchten"
model.Phase.RelPerm.GeomNames =     "domain"
model.Geom.domain.RelPerm.Alpha =    1.0
model.Geom.domain.RelPerm.N =        2.0

# Saturation
model.Phase.Saturation.Type =             "VanGenuchten"
model.Phase.Saturation.GeomNames =         "domain"
model.Geom.domain.Saturation.Alpha =        1.0
model.Geom.domain.Saturation.N =            3.0
model.Geom.domain.Saturation.SRes =         0.001
model.Geom.domain.Saturation.SSat =         1.0

#Mannings
model.Mannings.Type = "Constant"
model.Mannings.GeomNames = "domain"
model.Mannings.Geom.domain.Value = 0.0000044

## 5. Phases contaminants, gravity and wells

In [8]:
# Phases
model.Phase.Names = "water"
model.Phase.water.Density.Type = "Constant"
model.Phase.water.Density.Value = 1.0
model.Phase.water.Viscosity.Type = "Constant"
model.Phase.water.Viscosity.Value = 1.0
model.Phase.water.Mobility.Type = "Constant"
model.Phase.water.Mobility.Value = 1.0

# Contaminants
model.Contaminants.Names = ""

# Gravity
model.Gravity = 1.0

#Wells
model.Wells.Names = ""

# Phase Sources
model.PhaseSources.water.Type = "Constant"
model.PhaseSources.water.GeomNames = "domain"
model.PhaseSources.water.Geom.domain.Value = 0.0

## 6. Timing
The units of time are set by the *hydraulic conductivity*, $K$ units $[LT-1]$, in the case our units are *hours*. 

Here we want to simulate a simple rainstorm where we can turn rain on for x hours and off for y hours. 

To do this we setup two time cycles: `constant` and `rainrec`. 
- `constant` is a cycle that repeats for the entire simulation and we use this for any varialbes that we don't want to be changing with time. 
- `rainrec` is our time varying rainfall and recession tye cycle.  We give it two periods `rain` and `rec` and we define the length of `rain` to be 5 and the length of `rec` to be 20 and give it a repeat value of -1 indicating that this cycle should repeat for for as long as the simulation goes. 

In this exampe our `StopTime` is 20 hours so we will have 5 hours of the `rain` period and `15` hours of the `rec` period. 

NOTE: Our choice of naming the periods in this cycle `rain` and `rec` is just for our own purposes. The names do not have any inherent meaning here. We could have named them `cat` and `dog` if we wanted. What matters is that later on when we define the boundary conditions we use these names and associate fluxes with them. 

In [9]:
model.TimingInfo.BaseUnit = 1.0
model.TimingInfo.StartCount = 0
model.TimingInfo.StartTime = 0.0
model.TimingInfo.StopTime = 20.0
model.TimingInfo.DumpInterval = 1.0
model.TimeStep.Type = "Constant"
model.TimeStep.Value = 1.0

#Time cycles
model.Cycle.Names ="constant rainrec"

model.Cycle.constant.Names = "alltime"
model.Cycle.constant.alltime.Length = 1
model.Cycle.constant.Repeat = -1

model.Cycle.rainrec.Names = "rain rec"
model.Cycle.rainrec.rain.Length = 5
model.Cycle.rainrec.rec.Length = 20
model.Cycle.rainrec.Repeat = -1

## 7. Boundary and intial conditions

### 7.1 Boundary conditions
Now, we assign Boundary Conditions for each face (each of the Patches in the domain defined before). Recall the previously stated Patches and associate them with the boundary conditions that follow. The bottom and sides of our domain are all set to no-flow (i.e. constant flux of 0) boundaries. 

For the top boundary we are going to use the `rainrec` time cycle we defines above and apply a flux of -0.05 [m/hr] over the `rain` period and 0 over the `rec` period.  Note that we must use the period names that were defined for our time clcye. Also note that the flux value for `rain` is negative. This is beacause the z axix in parflow points up so if we want to rain on the domain we need a flux that is pointing down into the domain (i.e. in the negative z direction). 

NOTE: The units of this flux should be [1/T] so if you have a flux that is [L/T] remember to divide by the thickness of the layer you are applying it to (in this case our top layer)

In [10]:
model.BCPressure.PatchNames = "top bottom side"

model.Patch.bottom.BCPressure.Type = "FluxConst"
model.Patch.bottom.BCPressure.Cycle = "constant"
model.Patch.bottom.BCPressure.alltime.Value = 0.0

model.Patch.side.BCPressure.Type = "FluxConst"
model.Patch.side.BCPressure.Cycle = "constant"
model.Patch.side.BCPressure.alltime.Value = 0.0

model.Patch.top.BCPressure.Type = "OverlandKinematic"
model.Patch.top.BCPressure.Cycle = "rainrec"
model.Patch.top.BCPressure.rain.Value = -0.05
model.Patch.top.BCPressure.rec.Value = 0.0

### 7.2 Initial conditions: water pressure
Again our focus is on the respons of the land surface to precipitaiton here so we are not concerned about our groundwater configuration. To make things simple we will start with a pressure head of 1 relative to the bottom of the domain everywhere. 

In [11]:
model.ICPressure.Type = "HydroStaticPatch"
model.ICPressure.GeomNames = "domain"
model.Geom.domain.ICPressure.Value = 1.005
model.Geom.domain.ICPressure.RefGeom = "domain"
model.Geom.domain.ICPressure.RefPatch = "bottom"

## 8. Solver settings

### 8.1 Outputs:
Now we specify what outputs we would like written. In this example we specify that we would like to write out CLM variables as well as Pressure and Saturation. However, there are many options for this and you should change these options according to what type of analysis you will be performing on your results. A complete list of print options is provided in § 6.1.32.

In [12]:
model.Solver.PrintSubsurfData = True
model.Solver.PrintPressure = True
model.Solver.PrintSaturation = True
model.Solver.PrintMask = True
model.Solver.PrintVelocities = False

### 8.2 Solver parameters

In [13]:
# Solver types
model.Solver = "Richards"
model.Solver.TerrainFollowingGrid = True
model.Solver.Linear.Preconditioner = "PFMG"
model.Solver.Linear.Preconditioner.PCMatrixType = "FullJacobian"

# Exact solution
model.KnownSolution = "NoKnownSolution"

# Solver settings
model.Solver.MaxIter = 25000
model.Solver.Drop = 1e-20
model.Solver.AbsTol = 1e-8

model.Solver.MaxConvergenceFailures = 8
model.Solver.Nonlinear.MaxIter = 1000
model.Solver.Nonlinear.ResidualTol = 1e-6
model.Solver.Nonlinear.EtaChoice =  "EtaConstant"
model.Solver.Nonlinear.EtaValue = 0.001
model.Solver.Nonlinear.UseJacobian = True
model.Solver.Nonlinear.DerivativeEpsilon = 1e-16
model.Solver.Nonlinear.StepTol = 1e-15
model.Solver.Nonlinear.Globalization = "LineSearch"

model.Solver.Linear.KrylovDimension = 70
model.Solver.Linear.MaxRestarts = 2


## 9.  Distribute files, write the model and run the simulation

In [14]:
# Distribute input files
# slope files are 2D (i.e. they only have one layer) so you need to set NZ to 1 before you distribute them
# Make sure to set it back to your actual NZ before distributing 3D files or running your model
model.ComputationalGrid.NZ =1
model.dist("slopex_LW.pfb")
model.dist("slopey_LW.pfb")
model.ComputationalGrid.NZ =10

# write
model.write()
model.write(file_format='yaml')
model.write(file_format='json')

#run
model.run()


# ==============================================================================
# ParFlow directory
#  - /Users/reed/parflow/parflow.v3.10.0
# ParFlow version
#  - 3.10.0
# Working directory
#  - /Users/reed/Projects/parflow_python_shortcourse/exercises/little_washita/outputs/LW_ParkingLot
# ParFlow database
#  - LW_ParkingLot.pfidb
# ==============================================================================


# ==============================================================================
# ParFlow ran successfully 💦 💦 💦 
# ==============================================================================

